In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler
from tqdm.auto import tqdm

# 데이터 처리

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using {device} device")

Using cuda device


In [ ]:
time_series = pd.DataFrame()
target_label = pd.DataFrame()

In [ ]:
# 학습 데이터

path = '/content/drive/MyDrive/Colab_Notebooks/data/meta_data/train'
count = 0
row = pd.DataFrame()

for i in range(75):
    if i + 1 < 10:
        i_str = '0' + str(i +1)
    elif i + 1 > 10:
        i_str = str(i +1)
    elif i + 1 == 10:
        i_str = str(i + 1)

    file_list = os.listdir(path + '/CASE' + i_str)
    file_list_py = [file for file in file_list if file.endswith('.csv')]

    for l in file_list_py:
        data = pd.read_csv(path + '/CASE' + i_str + '/' + l)
        time_series = pd.concat([time_series, data])

In [ ]:
# 정답 데이터

path = '/content/drive/MyDrive/Colab_Notebooks/data/meta_data/target'
count2 = 0

for j in range(75):
    if j + 1 < 10:
        j_str = '0' + str(j +1)
    elif j + 1 > 10:
        j_str = str(j +1)
    elif j + 1 == 10:
        j_str = str(j + 1)

    file_list = os.listdir(path + '/CASE' + j_str)
    file_list_py = [file for file in file_list if file.endswith('.csv')]

    for k in file_list_py:
        target = pd.read_csv(path + '/CASE' + j_str + '/' + k)
        target_label = pd.concat([target_label, target])

### 데이터셋 변환

In [ ]:
time_dataset = time_series.to_numpy()
time_dataset = time_dataset[:, 1:]

target_dataset = target_label.to_numpy()
target_dataset = target_dataset[:, 1]

In [ ]:
print(len(time_dataset) / 1440)

1292.0


In [ ]:
def sliding_hours(data, target):
    x = []
    y = []

    for i in range(len(target)):
        x2 = []

        for j in range(24):
            _x = data[i][(j * 60): ((j+1) * 60)]
            array = []
            for k in range(18):
                _x1 = np.mean(_x[:, k])
                array.append(_x1)
            x2.append(np.array(array))

        x.append(np.array(x2))

    return np.array(x), target

In [ ]:
# 하루 단위로 데이터를 분리하여 label과 맵핑

def sliding_windows(data, target):
    x = []
    y = []

    for i in range(len(target)):
        _x = data[(i * 1440): ((i + 1) * 1440)]
        _y = target[i]
        x.append(_x)
        y.append(_y)

    return np.array(x),np.array(y)

# MinMaxScaler
sc_df = MinMaxScaler()
training_data = sc_df.fit_transform(time_dataset)

# 배치 사이즈
x, y = sliding_windows(training_data, target_dataset)

# x1, y1 = sliding_hours(x, y)


In [ ]:
train_size = int(len(y) * 0.7)
test_size = len(y) - train_size

trainX = Variable(torch.Tensor(np.array(x[:1020]))).to(device)
trainY = Variable(torch.Tensor(np.array(y[:1020]))).to(device)

testX = Variable(torch.Tensor(np.array(x[1020:1275]))).to(device)
testY = Variable(torch.Tensor(np.array(y[1020:1275]))).to(device)

In [ ]:
print(trainX.shape)
print(trainY.shape)
print(testX.shape)
print(testY.shape)

train_dataset = TensorDataset(trainX, trainY)
test_dataset = TensorDataset(testX, testY)

torch.Size([1020, 1440, 18])
torch.Size([1020])
torch.Size([255, 1440, 18])
torch.Size([255])


In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=20, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=5, shuffle=False)

In [ ]:
for x, y in train_dataloader:
    print(x.size(1))
    print(x[:,:,1].shape)
    print(x[:,:,1])
    print(y.shape)
    break

1440
torch.Size([20, 1440])
tensor([[0.1456, 0.1456, 0.1456,  ..., 0.1456, 0.1456, 0.1456],
        [0.2237, 0.2237, 0.2249,  ..., 0.2448, 0.2436, 0.2436],
        [0.2068, 0.2076, 0.2073,  ..., 0.2153, 0.2153, 0.2153],
        ...,
        [0.2862, 0.2862, 0.2838,  ..., 0.2802, 0.2790, 0.2790],
        [0.1071, 0.1071, 0.1071,  ..., 0.1158, 0.1203, 0.1232],
        [0.2225, 0.2237, 0.2273,  ..., 0.2177, 0.2189, 0.2189]],
       device='cuda:0')
torch.Size([20])


# 모델

### LSTM

In [ ]:
class LSTM(nn.Module):

    def __init__(self, input_size, hidden_size, num_layers, bidirectional):
        super(LSTM, self).__init__()
        
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.bidirectional = bidirectional
        
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True, bidirectional=bidirectional).to(device)
        
        self.fc = nn.Linear(hidden_size, 1).to(device)
        # self.fc2 = nn.Linear(256, 1).to(device)

    def forward(self, x):
        h_0 = Variable(torch.zeros(
            self.num_layers, x.size(1), self.hidden_size)).to(device)
        
        c_0 = Variable(torch.zeros(
            self.num_layers, x.size(1), self.hidden_size)).to(device)
        
        # Propagate input through LSTM
        ula, (h_out, _) = self.lstm(x, (h_0, c_0))
                
        out = self.fc(h_out.view(-1, self.hidden_size))
        # out = self.fc2(out)
        
        return out

### CNN-LSTM

In [ ]:
class MultiConvLstm(nn.Module):
    def __init__(self, n_features, n_hidden, seq_len, n_layers):
        super(MultiConvLstm, self).__init__()
        self.n_hidden = n_hidden
        self.seq_len = seq_len
        self.n_layers = n_layers

        self.c1 = nn.Conv1d(in_channels=1, out_channels=1, kernel_size = 2, stride = 1) # 1D CNN 레이어 추가
        
        self.lstm = nn.LSTM(
            input_size=n_features,
            hidden_size=n_hidden,
            num_layers=n_layers
        )
        self.linear = nn.Linear(in_features=n_hidden, out_features=1)

    def reset_hidden_state(self):
        self.hidden = (
            torch.zeros(self.n_layers, self.seq_len-1, self.n_hidden),
            torch.zeros(self.n_layers, self.seq_len-1, self.n_hidden)
        )

    def forward(self, x):
        
        for idx in range(n_features):
            

        sequences = self.c1(sequences.view(len(sequences), 1, -1))
        lstm_out, self.hidden = self.lstm(
            sequences.view(len(sequences), self.seq_len-1, -1),
            self.hidden
        )
        last_time_step = lstm_out.view(self.seq_len-1, len(sequences), self.n_hidden)[-1]
        y_pred = self.linear(last_time_step)
        return y_pred

### GRU

In [ ]:
# GRU

class GRUNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, drop_prob=0.2):
        super(GRUNet, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = 2
        
        self.gru = nn.GRU(input_dim, hidden_dim, n_layers, batch_first=True, dropout=drop_prob)
        self.fc = nn.Linear(hidden_dim, 1)
        self.relu = nn.ReLU()
        
    def forward(self, x, h):
        out, h = self.gru(x, h)
        out = self.fc(self.relu(out[:,-1]))
        return out, h
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device)
        return hidden

# 학습

In [ ]:
# validation 함수

def validation(model, vali_loader, criterion, device):
    model.eval()
    vali_loss = []

    with torch.no_grad():
        for idx, sample in enumerate(vali_loader):
            x_vali, y_vali = sample

            out = model(x_vali)
            loss = criterion(out, y_vali)
            vali_loss.append(loss.item())

    vali_mae_loss = np.mean(vali_loss)
    return vali_mae_loss

In [ ]:
# LSTM 학습

num_epochs = 50
learning_rate = 1e-3
optimizer_list = ['adam', 'rmsprop', 'sgd']

input_size = 18
hidden_size = 512
num_layers = 1

best_mae = 9999

model = LSTM(input_size, hidden_size, num_layers, bidirectional=False).to(device)

criterion = torch.nn.L1Loss().to(device)    # mean-squared error for regression
#optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)

# Train the model
for opti in optimizer_list:
    if opti == 'sgd':
        optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
    elif opti == 'adam':
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    elif opti == 'rmsprop':
        optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
    
    print('optimizer - {}'.format(opti))
    for epoch in range(num_epochs):
        model.train()
        train_loss = []
        for x_train, label in tqdm(iter(train_dataloader)):
            
            optimizer.zero_grad()
            outputs = model(x_train)
            out = torch.squeeze(outputs)
        
            # obtain the loss function
            loss = criterion(out, label)
        
            loss.backward()
        
            optimizer.step()

            train_loss.append(loss.item())

            # validation
        vali_mae = validation(model, test_dataloader, criterion, device)

        print(f'Epoch [{epoch}] Train MAE : [{np.mean(train_loss):.5f}] Validation MAE : [{vali_mae:.5f}]\n')

        if best_mae > vali_mae:
            best_mae = vali_mae
            if opti == 'sgd':
                torch.save(model.state_dict(), '/content/drive/MyDrive/Colab_Notebooks/competition/meta_LSTM_hour_length_opti_sgd.pth')
            elif opti == 'adam':
                torch.save(model.state_dict(), '/content/drive/MyDrive/Colab_Notebooks/competition/meta_LSTM_hour_length_opti_adam.pth')
            elif opti == 'rmsprop':
                torch.save(model.state_dict(), '/content/drive/MyDrive/Colab_Notebooks/competition/meta_LSTM_hour_length_opti_rmsprop.pth')
            print('------  Model Saved.  ---------')

optimizer - adam


  0%|          | 0/51 [00:00<?, ?it/s]

RuntimeError: ignored

In [ ]:
def train(train_loader, learn_rate, hidden_dim=256, EPOCHS=20, model_type="GRU"):
    
    # Setting common hyperparameters

    best_mae = 999
    

    # Instantiating the models
    if model_type == "GRU":
        model = GRUNet(input_dim, hidden_dim)
    elif model_type == "LSTM":
        model = LSTM(input_size, hidden_dim, num_layers, bidirectional)
    model.to(device)
    
    # Defining loss function and optimizer
    criterion = nn.L1Loss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learn_rate)
    
    model.train()
    print("Starting Training of {} model".format(model_type))
    epoch_times = []

    # Start training loop
    for epoch in range(1,EPOCHS+1):
        train_loss = []
        h = model.init_hidden(20)
        avg_loss = 0.
        counter = 0
        for x, label in tqdm(iter(train_loader)):
            counter += 1
            if model_type == "GRU":
                h = h.data
            else:
                h = tuple([e.data for e in h])

            model.zero_grad()
            
            out, h = model(x.to(device).float(), h)
            loss = criterion(out, label.to(device).float())

            loss.backward()

            optimizer.step()

            train_loss.append(loss.item())

        vali_mae = validation(model, test_dataloader, criterion, device)

        print(f'Epoch [{epoch}] Train MAE : [{np.mean(train_loss):.5f}] Validation MAE : [{vali_mae:.5f}]\n')

        if best_mae > vali_mae:
            best_mae = vali_mae
            torch.save(model.state_dict(), '/content/drive/MyDrive/Colab_Notebooks/competition/meta_gru.pth')
            print('model saved!')


# def evaluate(model, test_x, test_y, label_scalers):
#     model.eval()
#     outputs = []
#     targets = []
#     start_time = time.clock()
#     for i in test_x.keys():
#         inp = torch.from_numpy(np.array(test_x[i]))
#         labs = torch.from_numpy(np.array(test_y[i]))
#         h = model.init_hidden(inp.shape[0])
#         out, h = model(inp.to(device).float(), h)
#         outputs.append(label_scalers[i].inverse_transform(out.cpu().detach().numpy()).reshape(-1))
#         targets.append(label_scalers[i].inverse_transform(labs.numpy()).reshape(-1))
#     print("Evaluation Time: {}".format(str(time.clock()-start_time)))
#     sMAPE = 0
#     for i in range(len(outputs)):
#         sMAPE += np.mean(abs(outputs[i]-targets[i])/(targets[i]+outputs[i])/2)/len(outputs)
#     print("sMAPE: {}%".format(sMAPE*100))
#     return outputs, targets, sMAPE

# Run

In [ ]:
input_dim = next(iter(train_dataloader))[0].shape[2]
output_dim = 1
n_layers = 2

model = GRUNet(input_dim, hidden_dim=256).to(device)

# optimizer = torch.optim.SGD(params = model.parameters(), lr = 0.003])
scheduler = None

train(train_dataloader, learn_rate=0.05, hidden_dim=256, EPOCHS=20, model_type="GRU")

Starting Training of GRU model


  0%|          | 0/65 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:96: UserWarning: Using a target size (torch.Size([20])) that is different to the input size (torch.Size([20, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


RuntimeError: ignored